# download data

In [1]:
import kagglehub
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")
print("Kaggle路径:", path)

# 复制到你的目标目录
!mkdir -p /content/9517_data
!cp -r "{path}/." "/content/9517_data/"
!ls -R /content/9517_data


流式输出内容被截断，只能显示最后 5000 行内容。
snail-202-_jpg.rf.b6f1fd0780a89b080fec8afacdaacb61.txt
snail-205-_jpg.rf.53179db9b264995524ac9d4a3dafeb3e.txt
snail-205-_jpg.rf.5c76422229f9c6d0aee58ddbe17c7bdd.txt
snail-205-_jpg.rf.bf63fac5a41d621be789bf2c5867bed4.txt
snail-207-_jpg.rf.4d088fce33ffee487682d6f104737e2e.txt
snail-207-_jpg.rf.ca62becc9aabfbb0cb4fc3e3b63f4311.txt
snail-207-_jpg.rf.df199f155c88a67ae792a5630c79b0d2.txt
snail-209-_jpg.rf.965b52a5e84ad2fd61bc20b3f579b4a7.txt
snail-209-_jpg.rf.d30eff1932447cdbf7858bb798973ad4.txt
snail-209-_jpg.rf.e3d3407a7e8fb1120802940fbfaa3ae4.txt
snail-20-_jpg.rf.8c080af19e456f68f322dab5c709b36f.txt
snail-20-_jpg.rf.b4e6f388833a4b7add22d0ba56fc169c.txt
snail-20-_jpg.rf.b90ed73a975c0a5fcdf22560511b7d8a.txt
snail-210-_jpg.rf.24e07238190758c03ed6248668f78855.txt
snail-210-_jpg.rf.54ccf60073e11d6a292989a9e1d717de.txt
snail-210-_jpg.rf.c5500463a1776c33d38e67797a9b287a.txt
snail-211-_jpg.rf.190ba1e887cc08272dcfb293af2cc311.txt
snail-211-_jpg.rf.d55d7e60f7d9afe6fc31e8e

# Generate the .cls file required for resnet

In [2]:
# 跑resnet 所需的cls文件
# 1) 依赖
!pip -q install ultralytics opencv-python-headless pyyaml tqdm

# 2) 生成分类数据：从 YOLO 标注裁小图
import os, glob, yaml, cv2
from tqdm import tqdm

ROOT = "/content/9517_data"           # 就是你截图里的那个目录
DATA_YAML = os.path.join(ROOT, "data.yaml")
OUT = os.path.join(ROOT, "cls_data")  # 分类输出：ImageFolder 结构

with open(DATA_YAML, "r") as f:
    y = yaml.safe_load(f)

# names -> 类名列表
names = y["names"] if isinstance(y["names"], list) else [y["names"][k] for k in sorted(y["names"], key=lambda x:int(x))]

def yolo2xyxy(cx, cy, bw, bh, w, h):
    x1 = int((cx - bw/2) * w); y1 = int((cy - bh/2) * h)
    x2 = int((cx + bw/2) * w); y2 = int((cy + bh/2) * h)
    return max(0,x1), max(0,y1), min(w,x2), min(h,y2)

def ensure_dirs(split):
    for cls in names:
        os.makedirs(os.path.join(OUT, split, cls), exist_ok=True)

for split in ["train","valid","test"]:
    img_dir = os.path.join(ROOT, split, "images")
    lbl_dir = os.path.join(ROOT, split, "labels")
    if not (os.path.isdir(img_dir) and os.path.isdir(lbl_dir)):
        print(f"[{split}] 缺少 images/labels，跳过")
        continue
    ensure_dirs(split)
    for ip in tqdm(sorted(glob.glob(os.path.join(img_dir, "*"))), desc=f"裁剪 {split}"):
        base = os.path.splitext(os.path.basename(ip))[0]
        lp = os.path.join(lbl_dir, base + ".txt")
        if not os.path.exists(lp):
            continue
        img = cv2.imread(ip)
        if img is None:
            continue
        h, w = img.shape[:2]
        with open(lp, "r") as f:
            for i, line in enumerate([l.strip() for l in f if l.strip()]):
                ss = line.split()
                cls = int(ss[0]); cx, cy, bw, bh = map(float, ss[1:5])
                x1,y1,x2,y2 = yolo2xyxy(cx,cy,bw,bh,w,h)
                # 适度加一点 padding（10%），防止裁得过紧
                pw = int((x2-x1)*0.10); ph = int((y2-y1)*0.10)
                x1 = max(0, x1-pw); y1 = max(0, y1-ph)
                x2 = min(w, x2+pw); y2 = min(h, y2+ph)
                crop = img[y1:y2, x1:x2]
                if crop.size == 0:
                    continue
                save_dir = os.path.join(OUT, split, names[cls])
                cv2.imwrite(os.path.join(save_dir, f"{base}_{i}.jpg"), crop)

print("✅ 已生成分类数据到:", OUT)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.9 MB/s eta 0:00:00


裁剪 test: 100%|██████████| 546/546 [00:01<00:00, 323.20it/s]

✅ 已生成分类数据到: /content/9517_data/cls_data


# process
* find best parameters of yolo and resnet
* get the two-stage model result-acc
* get the yolo model result map 

In [7]:
import os, sys, subprocess, re

ROOT = "/content/9517_data"
os.makedirs(ROOT, exist_ok=True)
os.chdir(ROOT)
print("工作目录:", os.getcwd())

PY = sys.executable or "python3"

# 4 个步骤：YOLO 检测 -> 裁剪 -> ResNet 分类 -> 两阶段推理
STEPS = [
    # 第 1 步：YOLO 检测训练，开启超参数搜索 + 最后在 test 上评估一次
    (1, f'{PY} -u scripts/train_detect_yolo.py --hp_search --eval_test --epochs 50 --name yolo_ft'),

    # 第 2 步：根据 YOLO 标签裁剪出分类数据
    (2, f'{PY} -u scripts/crop_from_yolo_labels.py'),

    # 第 3 步：ResNet 分类器（内部自己做 head+finetune 和超参搜索）
    (3, f'{PY} -u scripts/train_resnet_classifier.py'),

    # 第 4 步：用最新的 YOLO best.pt + ResNet 分类器做两阶段推理
    (4, f'{PY} -u scripts/infer_two_stage.py'),
]

# 只刷新这一种行：[YOLO] ▶ progress: xx%
PROG_RE = re.compile(r'^\[YOLO\] ▶ progress:\s+\d+%')

def run(cmd):
    print(f'[RUN] "{cmd}"')
    p = subprocess.Popen(
        cmd,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        bufsize=1,
        text=True
    )
    last_progress = ""
    try:
        for line in p.stdout:
            if PROG_RE.match(line.strip()):
                # 进度行：单行刷新
                last_progress = line.strip()
                sys.stdout.write("\r" + last_progress)
                sys.stdout.flush()
            else:
                # 普通输出：如果之前有进度行，先换行
                if last_progress:
                    sys.stdout.write("\n")
                    sys.stdout.flush()
                    last_progress = ""
                sys.stdout.write(line)
                sys.stdout.flush()
    finally:
        p.wait()

    if p.returncode != 0:
        raise SystemExit(f"[FATAL] 子进程失败：{cmd} | returncode={p.returncode}")

def main():
    print(f"工作目录: {os.getcwd()}\n")
    for i, cmd in STEPS:
        print(f"[{i}/4] RUN: {cmd}")
        run(cmd)

if __name__ == "__main__":
    main()


工作目录: /content/9517_data
工作目录: /content/9517_data

[1/4] RUN: /usr/bin/python3 -u scripts/train_detect_yolo.py --hp_search --eval_test --epochs 50 --name yolo_ft
[RUN] "/usr/bin/python3 -u scripts/train_detect_yolo.py --hp_search --eval_test --epochs 50 --name yolo_ft"
[BOOT] enter train_detect_yolo.py
[BOOT] importing yaml ...
[BOOT] yaml OK
[BOOT] importing numpy ...
[BOOT] numpy OK
[BOOT] importing ultralytics ...
[BOOT] ultralytics OK
[BOOT] importing torch (this step may take a while on first run) ...
[BOOT] torch OK | version=2.8.0+cu126
[RUN] "/usr/bin/python3 -u scripts/train_detect_yolo.py"
[BOOT] importing data.yaml ...
[YOLO] data.yaml: {'train': '/content/9517_data/train/images', 'val': '/content/9517_data/valid/images', 'test': '/content/9517_data/test/images', 'nc': 12, 'names': ['Ants', 'Bees', 'Beetles', 'Caterpillars', 'Earthworms', 'Earwigs', 'Grasshoppers', 'Moths', 'Slugs', 'Snails', 'Wasps', 'Weevils']}
[YOLO] 数据摘要 | train: images=11502, labels=11502 | valid: image

# get the two-stage model map

In [8]:
import math
from pathlib import Path

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torchvision import models, transforms

from ultralytics import YOLO
import yaml
import warnings

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="You are using `torch.load` with `weights_only=False`"
)


# ================== 配置 ==================
YOLO_WEIGHTS = Path("yolo_best.pt")
CLS_WEIGHTS  = Path("resnet50_cls_best.pth")
DATA_YAML    = Path("data.yaml")

IOU_THRESH   = 0.5   # mAP@0.5
CONF_THRESH  = 0.25  # YOLO 检测阈值


# ================== 工具函数 ==================
def bbox_iou(box1, box2):
    """计算两个 bbox（xyxy）的 IoU"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_w = max(0.0, x2 - x1)
    inter_h = max(0.0, y2 - y1)
    inter = inter_w * inter_h

    area1 = max(0.0, (box1[2] - box1[0])) * max(0.0, (box1[3] - box1[1]))
    area2 = max(0.0, (box2[2] - box2[0])) * max(0.0, (box2[3] - box2[1]))
    union = area1 + area2 - inter + 1e-16

    return inter / union


def compute_ap(recall, precision):
    """对 PR 曲线做插值积分，计算 AP"""
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))

    # 单调递减包络
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = max(mpre[i - 1], mpre[i])

    idx = np.where(mrec[1:] != mrec[:-1])[0]

    ap = 0.0
    for i in idx:
        ap += (mrec[i + 1] - mrec[i]) * mpre[i + 1]
    return ap


# ================== 1. 读取 data.yaml ==================
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)

test_img_dir = Path(data_cfg["test"])            # e.g. test/images
test_label_dir = test_img_dir.parent / "labels"
class_names = data_cfg.get("names", None)
num_classes = len(class_names) if class_names is not None else None

print(f"[INFO] Test images dir : {test_img_dir}")
print(f"[INFO] Test labels dir : {test_label_dir}")
print(f"[INFO] Num classes     : {num_classes}")


# ================== 2. 加载 YOLO 检测器 ==================
print("[INFO] Loading YOLO detector...")
det_model = YOLO(str(YOLO_WEIGHTS))


# ================== 3. 加载 ResNet 分类器 ==================
print("[INFO] Loading ResNet classifier...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ckpt = torch.load(CLS_WEIGHTS, map_location=device)

# 如果 checkpoint 里带有 classes，就用它覆盖 data.yaml 的 names
if isinstance(ckpt, dict) and "classes" in ckpt:
    class_names = ckpt["classes"]
    num_classes = len(class_names)
    print(f"[INFO] classes loaded from checkpoint: {class_names}")

if num_classes is None:
    raise ValueError("num_classes 未能从 data.yaml 或 checkpoint 中解析，请检查。")

cls_model = models.resnet50(weights=None)
in_features = cls_model.fc.in_features
cls_model.fc = nn.Linear(in_features, num_classes)

if isinstance(ckpt, dict) and "state_dict" in ckpt:
    state_dict = ckpt["state_dict"]
else:
    state_dict = ckpt

cls_model.load_state_dict(state_dict)
cls_model.to(device)
cls_model.eval()
print("[INFO] ResNet classifier loaded.")

# 和训练尽量一致的 transform（如果你训练脚本不同，按训练那边改）
cls_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


# ================== 4. 读取测试集的图像信息和 GT ==================
print("[INFO] Collecting test images and ground truth boxes...")

img_size_dict = {}   # id -> (W, H)
img_ext_dict  = {}   # id -> suffix

for img_path in sorted(test_img_dir.glob("*.*")):
    if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
        continue
    image_id = img_path.stem
    with Image.open(img_path) as im:
        w, h = im.size
    img_size_dict[image_id] = (w, h)
    img_ext_dict[image_id] = img_path.suffix

# gt_dict[cls][image_id] = [box1, box2, ...]，box 为像素级 xyxy
gt_dict = {c: {} for c in range(num_classes)}
gt_count_per_class = np.zeros(num_classes, dtype=int)

for image_id, (w, h) in img_size_dict.items():
    label_path = test_label_dir / f"{image_id}.txt"
    if not label_path.exists():
        continue

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls_id = int(parts[0])
            xc, yc, bw, bh = map(float, parts[1:])

            x_center = xc * w
            y_center = yc * h
            bw_pixel = bw * w
            bh_pixel = bh * h

            x1 = x_center - bw_pixel / 2.0
            y1 = y_center - bh_pixel / 2.0
            x2 = x_center + bw_pixel / 2.0
            y2 = y_center + bh_pixel / 2.0

            box = np.array([x1, y1, x2, y2], dtype=float)

            if image_id not in gt_dict[cls_id]:
                gt_dict[cls_id][image_id] = []
            gt_dict[cls_id][image_id].append(box)
            gt_count_per_class[cls_id] += 1

print("[INFO] Ground truth loaded.")
print("      GT per class:", gt_count_per_class.tolist())


# ================== 5. Two-stage 推理 ==================
print("[INFO] Running two-stage inference on test set...")

all_predictions = []   # 每个元素: {"image_id", "cls", "conf", "box"}

image_ids_sorted = sorted(img_size_dict.keys())

for image_id in image_ids_sorted:
    img_path = test_img_dir / f"{image_id}{img_ext_dict[image_id]}"

    # YOLO 检测
    results = det_model(str(img_path), conf=CONF_THRESH, verbose=False)[0]

    if results.boxes is None or len(results.boxes) == 0:
        continue

    with Image.open(img_path).convert("RGB") as im:
        for box_tensor, conf_tensor in zip(results.boxes.xyxy, results.boxes.conf):
            x1, y1, x2, y2 = box_tensor.tolist()
            conf = float(conf_tensor.item())

            crop = im.crop((x1, y1, x2, y2))
            inp = cls_transform(crop).unsqueeze(0).to(device)

            with torch.no_grad():
                logits = cls_model(inp)
                pred_cls = int(torch.argmax(logits, dim=1).item())

            all_predictions.append({
                "image_id": image_id,
                "cls": pred_cls,
                "conf": conf,
                "box": np.array([x1, y1, x2, y2], dtype=float),
            })

print(f"[INFO] Total predictions: {len(all_predictions)}")


# ================== 6. 计算 mAP@0.5 ==================
print("[INFO] Computing mAP@0.5 ...")

aps = []

for cls_id in range(num_classes):
    gt_count = gt_count_per_class[cls_id]
    if gt_count == 0:
        aps.append(float("nan"))
        continue

    cls_preds = [p for p in all_predictions if p["cls"] == cls_id]
    if len(cls_preds) == 0:
        aps.append(0.0)
        name = class_names[cls_id] if class_names is not None else str(cls_id)
        print(f"Class {cls_id:02d} ({name:>12}): AP = 0.0000 (no predictions)")
        continue

    cls_preds.sort(key=lambda x: x["conf"], reverse=True)

    gt_matched = {img_id: np.zeros(len(gt_dict[cls_id].get(img_id, [])), dtype=bool)
                  for img_id in gt_dict[cls_id].keys()}

    tp = np.zeros(len(cls_preds))
    fp = np.zeros(len(cls_preds))

    for i, pred in enumerate(cls_preds):
        image_id = pred["image_id"]
        pred_box = pred["box"]

        gts = gt_dict[cls_id].get(image_id, [])
        if len(gts) == 0:
            fp[i] = 1
            continue

        ious = [bbox_iou(pred_box, gt_box) for gt_box in gts]
        best_iou = max(ious)
        best_gt_idx = int(np.argmax(ious))

        if best_iou >= IOU_THRESH and not gt_matched[image_id][best_gt_idx]:
            tp[i] = 1
            gt_matched[image_id][best_gt_idx] = True
        else:
            fp[i] = 1

    tp_cum = np.cumsum(tp)
    fp_cum = np.cumsum(fp)

    recalls = tp_cum / (gt_count + 1e-16)
    precisions = tp_cum / (tp_cum + fp_cum + 1e-16)

    ap = compute_ap(recalls, precisions)
    aps.append(ap)

    name = class_names[cls_id] if class_names is not None else str(cls_id)
    print(f"Class {cls_id:02d} ({name:>12}): AP = {ap:.4f}")

valid_aps = [ap for ap, cnt in zip(aps, gt_count_per_class)
             if cnt > 0 and not math.isnan(ap)]
mAP_50 = float(np.mean(valid_aps)) if len(valid_aps) > 0 else 0.0

print("=" * 60)
print(f"Two-stage mAP@0.5 = {mAP_50:.4f}")
print("=" * 60)


[INFO] Test images dir : test/images
[INFO] Test labels dir : test/labels
[INFO] Num classes     : 12
[INFO] Loading YOLO detector...
[INFO] Loading ResNet classifier...
[INFO] classes loaded from checkpoint: ['Ants', 'Bees', 'Beetles', 'Caterpillars', 'Earthworms', 'Earwigs', 'Grasshoppers', 'Moths', 'Slugs', 'Snails', 'Wasps', 'Weevils']
[INFO] ResNet classifier loaded.
[INFO] Collecting test images and ground truth boxes...
[INFO] Ground truth loaded.
      GT per class: [87, 44, 44, 93, 40, 73, 55, 47, 51, 50, 47, 58]
[INFO] Running two-stage inference on test set...
[INFO] Total predictions: 663
[INFO] Computing mAP@0.5 ...
Class 00 (        Ants): AP = 0.4864
Class 01 (        Bees): AP = 0.7658
Class 02 (     Beetles): AP = 0.3787
Class 03 (Caterpillars): AP = 0.2217
Class 04 (  Earthworms): AP = 0.2937
Class 05 (     Earwigs): AP = 0.5367
Class 06 (Grasshoppers): AP = 0.4606
Class 07 (       Moths): AP = 0.9085
Class 08 (       Slugs): AP = 0.4286
Class 09 (      Snails): AP = 